# Initialization

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
# Import packages
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
# Create the folder for saving files (if it doesn't exist)
#FOLDER = "Data/"
#os.makedirs(FOLDER, exist_ok=True)
#os.listdir(FOLDER)

In [3]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"Creating {JSON_FILE} for API results for {YEAR}.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)  

In [4]:
# Define function to write to json file
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [5]:
# Define a function that gets the movie with rating from TMDB database through its API
def get_movie_with_rating(movie_id):
    """ Adapted from source = https://github.com/celiao/tmdbsimple """
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # Save the .info and .releases dictionary
    info = movie.info()
    releases = movie.releases()
    # Loop through the countries in releases
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## Save a certification key in info with the certification
            info['certification'] = c['certification']
    return info

# Load Reference Data

In [6]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/preprocessed-title-basics.csv')

In [7]:
# The year of movies to be retrieved
YEAR = 2010
# Create an errors variable 
errors = [ ]

In [8]:
#Saving new year as the current df
df = basics.loc[ basics['start_year'] == YEAR].copy()
# saving movie ids to separate variable
movie_ids = df['tconst']

# JSON File

In [9]:
#Defining the JSON file to store results for year
JSON_FILE = f'tmdb_api_results_{YEAR}.json'
print(f'data will be saved to: {JSON_FILE}')

data will be saved to: tmdb_api_results_2010.json


In [10]:
# Create a JSON file
create_json_file(JSON_FILE,  delete_if_exists=False)

[i] tmdb_api_results_2010.json already exists.


In [11]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

In [12]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# API Call

In [13]:
# Using API Credentials
with open('/Users/kristansimbulan/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [14]:
# Set API_KEY variable to the user's api-key 
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [15]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    #Get index and movie id from list
    try:
        # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    except Exception as e:
        errors.append([movie_id, e])
        
print(f"- Total errors: {len(errors)}")

Movies from 2010:   0%|          | 0/1115 [00:00<?, ?it/s]

- Total errors: 1115


In [16]:
# Save the year's result as csv.gz file
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)